<a href="https://colab.research.google.com/github/DonaldTurton/AT2_Advanced_Data_Algorithms/blob/feature-engineering/AT2_GetData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Predicting Energy Prices

In [0]:
# Packages
import pandas as pd
import numpy as np
from google.colab import files

# Libraries for XGBOOST
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer


In [2]:
# Read csv
url = 'https://raw.githubusercontent.com/DonaldTurton/AT2_Advanced_Data_Algorithms/master/ADAA_data.csv'
data = pd.read_csv(url)
data.head()

,Unnamed: 0,REGION,Year,Month,Day,time,TOTALDEMAND,RRP,PERIODTYPE,date_keep,Product.code.x,Bureau.of.Meteorology.station.number.x,Daily.global.solar.exposure..MJ.m.m.,Product.code.y,Bureau.of.Meteorology.station.number.y,Maximum.temperature..Degree.C.,Days.of.accumulation.of.maximum.temperature,Quality.x,Product.code.x.x,Bureau.of.Meteorology.station.number.x.x,Minimum.temperature..Degree.C.,Days.of.accumulation.of.minimum.temperature,Quality.y,Product.code.y.y,Bureau.of.Meteorology.station.number.y.y,Rainfall.amount..millimetres.,Period.over.which.rainfall.was.measured..days.,Quality
0,1,NSW1,2018,1,1,00:30:00,6912.25,91.86,TRADE,2018/01/01,IDCJAC0016,66024.0,31.0,IDCJAC0010,66037.0,28.3,1.0,N,IDCJAC0011,66037.0,20.8,1.0,N,IDCJAC0009,66037.0,0.0,1.0,N
1,2,NSW1,2018,1,1,01:00:00,6745.63,88.83,TRADE,2018/01/01,IDCJAC0016,66024.0,31.0,IDCJAC0010,66037.0,28.3,1.0,N,IDCJAC0011,66037.0,20.8,1.0,N,IDCJAC0009,66037.0,0.0,1.0,N
2,3,NSW1,2018,1,1,01:30:00,6466.06,73.62,TRADE,2018/01/01,IDCJAC0016,66024.0,31.0,IDCJAC0010,66037.0,28.3,1.0,N,IDCJAC0011,66037.0,20.8,1.0,N,IDCJAC0009,66037.0,0.0,1.0,N
3,4,NSW1,2018,1,1,02:00:00,6279.80,71.49,TRADE,2018/01/01,IDCJAC0016,66024.0,31.0,IDCJAC0010,66037.0,28.3,1.0,N,IDCJAC0011,66037.0,20.8,1.0,N,IDCJAC0009,66037.0,0.0,1.0,N
4,5,NSW1,2018,1,1,02:30:00,6177.31,69.27,TRADE,2018/01/01,IDCJAC0016,66024.0,31.0,IDCJAC0010,66037.0,28.3,1.0,N,IDCJAC0011,66037.0,20.8,1.0,N,IDCJAC0009,66037.0,0.0,1.0,N


In [3]:
# Exploring the data set
data.shape   #28280Obs 28var


(28280, 28)

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28280 entries, 0 to 28279
Data columns (total 28 columns):
Unnamed: 0                                        28280 non-null int64
REGION                                            28224 non-null object
Year                                              28280 non-null int64
Month                                             28280 non-null int64
Day                                               28280 non-null int64
time                                              28280 non-null object
TOTALDEMAND                                       28223 non-null float64
RRP                                               28223 non-null float64
PERIODTYPE                                        28223 non-null object
date_keep                                         28280 non-null object
Product.code.x                                    27696 non-null object
Bureau.of.Meteorology.station.number.x            27696 non-null float64
Daily.global.solar.exposure.

In [0]:
# Total demand distribution
data.TOTALDEMAND.describe()

count    28223.000000
mean      8083.738118
std       1236.545692
min       5552.230000
25%       7190.075000
50%       7998.070000
75%       8792.755000
max      13700.900000
Name: TOTALDEMAND, dtype: float64

In [0]:
data.RRP.describe()

count    28223.000000
mean        84.265707
std         58.661135
min          2.980000
25%         62.285000
50%         74.890000
75%         94.780000
max       4631.980000
Name: RRP, dtype: float64

In [0]:
#Find Null values 
data.isnull().sum()

Unnamed: 0                                            0
REGION                                               56
Year                                                  0
Month                                                 0
Day                                                   0
time                                                  0
TOTALDEMAND                                          57
RRP                                                  57
PERIODTYPE                                           57
date_keep                                             0
Product.code.x                                      584
Bureau.of.Meteorology.station.number.x              584
Daily.global.solar.exposure..MJ.m.m.                584
Product.code.y                                      584
Bureau.of.Meteorology.station.number.y              584
Maximum.temperature..Degree.C.                      584
Days.of.accumulation.of.maximum.temperature         584
Quality.x                                       

# **Feature Engineering**

In [4]:
# Create a price 1hr moving average

data = data.assign(RRP_1hr_moving_avg = data.RRP.rolling(window=2).mean())


# Create a price 2hr moving average

data = data.assign(RRP_2hr_moving_avg = data.RRP.rolling(window=4).mean())


# Create a demand 1hr moving average

data = data.assign(demand_1hr_moving_avg = data.TOTALDEMAND.rolling(window=2).mean())


# Create a demand 2hr moving average

data = data.assign(demand_2hr_moving_avg = data.TOTALDEMAND.rolling(window=4).mean())


# Ensure all of the new variables are setup correctly in the table. 

data.head()


,Unnamed: 0,REGION,Year,Month,Day,time,TOTALDEMAND,RRP,PERIODTYPE,date_keep,Product.code.x,Bureau.of.Meteorology.station.number.x,Daily.global.solar.exposure..MJ.m.m.,Product.code.y,Bureau.of.Meteorology.station.number.y,Maximum.temperature..Degree.C.,Days.of.accumulation.of.maximum.temperature,Quality.x,Product.code.x.x,Bureau.of.Meteorology.station.number.x.x,Minimum.temperature..Degree.C.,Days.of.accumulation.of.minimum.temperature,Quality.y,Product.code.y.y,Bureau.of.Meteorology.station.number.y.y,Rainfall.amount..millimetres.,Period.over.which.rainfall.was.measured..days.,Quality,RRP_1hr_moving_avg,RRP_2hr_moving_avg,demand_1hr_moving_avg,demand_2hr_moving_avg
0,1,NSW1,2018,1,1,00:30:00,6912.25,91.86,TRADE,2018/01/01,IDCJAC0016,66024.0,31.0,IDCJAC0010,66037.0,28.3,1.0,N,IDCJAC0011,66037.0,20.8,1.0,N,IDCJAC0009,66037.0,0.0,1.0,N,NaN,NaN,NaN,NaN
1,2,NSW1,2018,1,1,01:00:00,6745.63,88.83,TRADE,2018/01/01,IDCJAC0016,66024.0,31.0,IDCJAC0010,66037.0,28.3,1.0,N,IDCJAC0011,66037.0,20.8,1.0,N,IDCJAC0009,66037.0,0.0,1.0,N,90.345,NaN,6828.940,NaN
2,3,NSW1,2018,1,1,01:30:00,6466.06,73.62,TRADE,2018/01/01,IDCJAC0016,66024.0,31.0,IDCJAC0010,66037.0,28.3,1.0,N,IDCJAC0011,66037.0,20.8,1.0,N,IDCJAC0009,66037.0,0.0,1.0,N,81.225,NaN,6605.845,NaN
3,4,NSW1,2018,1,1,02:00:00,6279.80,71.49,TRADE,2018/01/01,IDCJAC0016,66024.0,31.0,IDCJAC0010,66037.0,28.3,1.0,N,IDCJAC0011,66037.0,20.8,1.0,N,IDCJAC0009,66037.0,0.0,1.0,N,72.555,81.4500,6372.930,6600.935
4,5,NSW1,2018,1,1,02:30:00,6177.31,69.27,TRADE,2018/01/01,IDCJAC0016,66024.0,31.0,IDCJAC0010,66037.0,28.3,1.0,N,IDCJAC0011,66037.0,20.8,1.0,N,IDCJAC0009,66037.0,0.0,1.0,N,70.380,75.8025,6228.555,6417.200


# XG Boost (First Attempt)

In [0]:
# Data Set Selection  (delete useless variable)
# data = data.loc[:, [-"Product.code.x", -"Product.code.y"]]

In [0]:
#Drops Na
data.dropna(axis=0, subset=['RRP'], inplace=True)

In [0]:
#Chek 
data.isnull().sum()

Unnamed: 0                                            0
REGION                                                0
Year                                                  0
Month                                                 0
Day                                                   0
time                                                  0
TOTALDEMAND                                           0
RRP                                                   0
PERIODTYPE                                            0
date_keep                                             0
Product.code.x                                      527
Bureau.of.Meteorology.station.number.x              527
Daily.global.solar.exposure..MJ.m.m.                527
Product.code.y                                      527
Bureau.of.Meteorology.station.number.y              527
Maximum.temperature..Degree.C.                      527
Days.of.accumulation.of.maximum.temperature         527
Quality.x                                       

In [0]:
# Data partition
y = data.RRP
X = data.drop(['RRP'], axis=1).select_dtypes(exclude=['object'])
train_X, test_X, train_y, test_y = train_test_split(X.as_matrix(), y.as_matrix(), test_size=0.25)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
#Imputer
my_imputer = Imputer()
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
# Build model
from xgboost import XGBRegressor

my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(train_X, train_y, verbose=False)

[04:13:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [0]:
# make predictions
predictions = my_model.predict(test_X)

from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))

Mean Absolute Error : 15.305550318715794
